In [ ]:
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')

import sys
sys.path.insert(0, "../")

import pandas as pd 
import numpy as np
import torch
from torchsummary import summary

torch.cuda.is_available()

# Building Block: Temporal Convolution

Below you find some sample code to test the temporal convolution. For verification you can use the example tensor some predefined weights.

In general, the `Conv2D` function from PyTorch requires some tensor with the dimensions `(N, C_in, H_in, W_in)`. In our case those dimensions corresponds to `(batch, features, nodes, time_steps)`.

In [ ]:
################################################
# Create example tensor                        #
# Either use a predefined tensor (uncomment)   #
# or use a random generated tensor (commented) #
################################################

# batch = 1
# features = 4
# time_steps = 6
# nodes = 3
# X = torch.randint(0, 10, (batch, features, nodes, time_steps)).type('torch.FloatTensor').cpu()

# Predefined example tensor for verification
X = torch.tensor([[
       [[1, 2, 1, 1],  # f1, n1 , t1 and t2 and t3 and t4
        [3, 4, 1, 1],  # f1, n2 , t1 and t2 and t3 and t4
        [2, 1, 1, 1]], # f1, n3 , t1 and t2 and t3 and t4
       [[2, 2, 1, 1],  # f2, n1 , t1 and t2 and t3 and t4
        [2, 2, 1, 1],  # f2, n2 , t1 and t2 and t3 and t4
        [2, 4, 1, 1]]  # f2, n3 , t1 and t2 and t3 and t4
     ]]).type('torch.FloatTensor').cpu()

X.shape
# x, y, z;Old: features, nodes, time; required: fearures, time, nodes
#X.permute(0,3,2,1)

In [ ]:
###############################
# Create Temporal Convolution #
###############################
import sys
sys.path.insert(0, "../")

from model.temporal_info_graph import TemporalConvolution

# Custom kernel! Dimension (c_out, c_in, kernel[0], kernel[1])
weights = torch.ones_like(torch.randint(0, 10, (5, 2, 1, 2)).type('torch.FloatTensor'))

# c_in = features, c_out can be chosen, kernel = (1, x), where x is free number less than the number of time steps.
# Weights and activation are optional.
tempConv = TemporalConvolution(c_in=2, c_out=2, kernel=(1, 3))#, weights=weights, activation=None).cpu()
print(X.shape, tempConv(X).shape)
H = tempConv(X)

# Building Block: Spectral Convolution

Reference implementations: https://github.com/FelixOpolka/STGCN-PyTorch/blob/master/stgcn.py

In [ ]:
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')


import torch
import numpy as np 

# Predefined example tensor for verification
X = torch.tensor([[
       [[1, 2, 1, 1],  # f1, n1 , t1 and t2 and t3 and t4
        [3, 4, 1, 1],  # f1, n2 , t1 and t2 and t3 and t4
        [2, 1, 1, 1]], # f1, n3 , t1 and t2 and t3 and t4
       [[2, 2, 1, 1],  # f2, n1 , t1 and t2 and t3 and t4
        [2, 2, 1, 1],  # f2, n2 , t1 and t2 and t3 and t4
        [2, 4, 1, 1]]  # f2, n3 , t1 and t2 and t3 and t4
     ]]).type('torch.FloatTensor').cpu()

A = torch.tensor([
    [0, 1, 0],
    [1, 0, 1], 
    [0, 1, 0]
    ]).type('torch.FloatTensor').cpu()

D = torch.diag(torch.sum(A, dim=0))
L = D - A

`torch.einsum(equation, *operands)`: This function provides a way of computing multilinear expressions (i.e. sums of products) using the Einstein summation convention.

**Equation**
* The left hand side lists the operands dimensions, separated by commas. There should be one index letter per tensor dimension. 
* The right hand side follows after -> and gives the indices for the output


In [ ]:
#################################################################################
# Check/Validation: This einsum corresponds to Adjacency multiplication in time #
#################################################################################

# lfs = torch.einsum("ij,jklm->kilm", [A, X.permute(3, 0, 2, 1)])
lfs = torch.einsum("ij,jklm->kilm", [A, X.permute(2, 0, 3, 1)])

print(lfs)#.permute(0, 2, 1, 3))
# Equal to A@X.permute(0,3,2,1)[0][i], i.e. multiplying the adjacency with the feature matrix at each time step.
# print(A@X.permute(0,3,2,1)[0][0])

In [ ]:
###############################
# Create Spectral Convolution #
###############################
import sys
sys.path.insert(0, "../")

from model.temporal_info_graph import SpectralConvolution

# To double check
weights = torch.eye(2).type('torch.FloatTensor')

# Weights and activation are optional.
specConv = SpectralConvolution(c_in=2, c_out=1, weights=weights, activation = None).cpu()
print(X.shape, specConv(X, A).shape, X.permute(0, 2, 1, 3).shape)
specConv(X, A)

# TemporalInfoGraph: Stacked temporal and spectral convolution.

In [ ]:
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')

import torch
import numpy as np 

# Predefined example tensor for verification
X = torch.tensor([[
       [[1, 2, 1, 1],  # f1, n1 , t1 and t2 and t3 and t4
        [3, 4, 1, 1],  # f1, n2 , t1 and t2 and t3 and t4
        [2, 1, 1, 1]], # f1, n3 , t1 and t2 and t3 and t4
       [[2, 2, 1, 1],  # f2, n1 , t1 and t2 and t3 and t4
        [2, 2, 1, 1],  # f2, n2 , t1 and t2 and t3 and t4
        [2, 4, 1, 1]]  # f2, n3 , t1 and t2 and t3 and t4
     ]]).type('torch.FloatTensor').cpu()

A = torch.tensor([
    [0, 1, 0],
    [1, 0, 1], 
    [0, 1, 0]
    ]).type('torch.FloatTensor').cpu()

D = torch.diag(torch.sum(A, dim=0))
L = D - A

In [ ]:
import sys
sys.path.insert(0, "../")

from model.temporal_info_graph import TemporalInfoGraph

# dim_in corresponds to (H, W), here (nodes, time steps)
tig = TemporalInfoGraph(c_in=2, c_out=6, spec_out=9, out=7, dim_in=(3,4), tempKernel=3)
gbl, lcl  = tig(X, A)
print(X.shape)
print(gbl.shape)
# print(gbl)
print(lcl.shape)
# print(lcl)

In [ ]:
gbl.mean(dim=1).shape

# Data Investigation

In [ ]:
######################
# Data Investigation #
######################
import sys
sys.path.insert(0, "../")

from data.preprocessing import create_skeleton

df_val = create_skeleton(folder="../../../Datasets/Kinetics-skeleton/kinetics-skeleton/kinetics_val/", cat="val", lim=10, to_mongoDB=False)

In [ ]:
from visualization.animation import animate_skeleton

# It can be that we don't have a skeleton in each frame.
animate_skeleton(df_val.iloc[2], lim_frames=None)

In [ ]:
############################## 
# Example: Read from MongoDB #
##############################
import pymongo

db_url = ""
with open("../.mongoURL") as f:
        db_url = f.readlines()       

client = pymongo.MongoClient(db_url)

collection = client.temporal_info_graph["kinect-skeleton"]

cursor = collection.find({})
for document in cursor:
        doc = pd.DataFrame([document])
        break

# Get feature matrix for sample 1 and frame 1
X = np.asarray(doc.iloc[0]["frames"][0]["feature_matrices"][0])

In [ ]:
#####################################
# Visualize Feature Matrix (Static) #
#####################################
import numpy as np 
import networkx as nx 

from data import KINECT_ADJACENCY

G = nx.Graph(KINECT_ADJACENCY)
nx.draw(G, [(x,y) for x,y in X], node_size=50,with_labels=True)